In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
# class Autoencoder(nn.Module):
#   def __init__(self, input_size, hidden_size, latent_size):
#     super(Autoencoder, self).__init__()
#     self.encoder = nn.Sequential(
#       nn.Linear(input_size, hidden_size),
#       nn.ReLU(), # should decide on the relu variant, but it's just basic relu rn as a placeholder
#       nn.Linear(hidden_size, latent_size),
#       nn.ReLU()
#     )
#     self.decoder = nn.Sequential(
#       nn.Linear(latent_size, hidden_size),
#       nn.ReLU(),
#       nn.Linear(hidden_size, input_size),
#       nn.ReLU()
#     )

#   def forward(self, x):
#     x = self.encoder(x)
#     x = self.decoder(x)
#     return x


In [ ]:
# # define dimensions
# input_size = 10000  # replace with correct number of features
# hidden_size = 10
# latent_size = 4

# # make model, instance of autoencoder
# model = Autoencoder(input_size, hidden_size, latent_size)

# # binary cross entropy (since output is in [0, 1]) <-- nope because it isn't, whoops
# loss_fn = nn.MSELoss()

# # optimizer (using adam for rn just bc i've used it before). will probably have to change to (mini batch?) LBGFS when using GPUs for parallelizing. refer to felix's big autoencoder paper
# optimizer = optim.Adam(model.parameters(), lr=0.001) # how/where do i set batch size

# # random data - replace with actual matrix
# data = np.random.rand(1700, 10000)  # 100 samples, 50 features (corresponds to input_size)

# # convert to tensor so it can be inputted
# data = torch.tensor(data, dtype=torch.float)

# # train model
# num_epochs = 100
# for epoch in range(num_epochs):
#   # forward pass
#   outputs = model(data)
#   loss = loss_fn(outputs, data)

#   # backprop
#   optimizer.zero_grad()
#   loss.backward()
#   optimizer.step()

#   # so we can see epochs and loss at each
#   if (epoch+1) % 10 == 0:
#     print('Epoch:', epoch+1, '/', num_epochs, ' Loss:', loss.item())

# # try to reconstruct the first sample based on the model ^
# sample_input = data[0].unsqueeze(0)
# reconstructed = model(sample_input)
# print("original:")
# print(sample_input)
# print("reconstructed:")
# print(reconstructed)

Epoch: 10 / 100  Loss: 0.2577957212924957
Epoch: 20 / 100  Loss: 0.25031405687332153
Epoch: 30 / 100  Loss: 0.2428545355796814
Epoch: 40 / 100  Loss: 0.23592348396778107
Epoch: 50 / 100  Loss: 0.2296619862318039
Epoch: 60 / 100  Loss: 0.22411970794200897
Epoch: 70 / 100  Loss: 0.2192796766757965
Epoch: 80 / 100  Loss: 0.21508640050888062
Epoch: 90 / 100  Loss: 0.21151889860630035
Epoch: 100 / 100  Loss: 0.20851656794548035
original:
tensor([[0.8810, 0.6215, 0.5171,  ..., 0.9816, 0.2788, 0.5191]])
reconstructed:
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.5049, 0.0000, 0.4936]],
       grad_fn=<ReluBackward0>)


In [ ]:
class Autoencoder(nn.Module):
  def __init__(self, input_size, hidden_size, latent_size):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
      nn.Linear(input_size, hidden_size),
      nn.ReLU(), # should decide on the relu variant, but it's just basic relu rn as a placeholder
      nn.Linear(hidden_size, latent_size),
      nn.ReLU()
    )
    self.decoder = nn.Sequential(
      nn.Linear(latent_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, input_size),
      nn.ReLU()
    )

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x

In [ ]:
# check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # to run on GPU on colab, go to runtime > change runtime type > T4 GPU

# define dimensions
input_size = 10000
hidden_size = 10
latent_size = 4

# make model, instance of autoencoder
model = Autoencoder(input_size, hidden_size, latent_size).to(device)

# binary cross entropy (since output is in [0, 1])
loss_fn = nn.MSELoss()

# optimizer
# actually not going to change this to LBGFS because it takes more memory (stores the hess) and rn memory is more of an issue than speed I think
optimizer = optim.Adam(model.parameters(), lr=0.001) # can tune lr, also not high priority. Should it be .01 instead of .001?

# random data - replace with actual matrix
data = np.random.rand(1700, 10000)

# convert to tensor so it can be inputted
data = torch.tensor(data, dtype=torch.float).to(device)

# create dataset
dataset = TensorDataset(data, data) # the first arg is the input, the second is the target. for autoencoder they're the same so it's just (data, data)

# create dataloader
batch_size = 64 # will probably be another thing to tune, but doubt it's high priority
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# train model
num_epochs = 100
for epoch in range(num_epochs):
  epoch_loss = 0.0
  for batch_data, _ in dataloader: # _ was for target but since it's the same thing might as well save mem
    batch_data = batch_data.to(device)

    # forward pass
    outputs = model(batch_data)
    loss = loss_fn(outputs, batch_data)
    epoch_loss += loss.item()

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # print epoch loss
  if (epoch+1) % 10 == 0:
    print('Epoch:', epoch+1, '/', num_epochs, ' Loss:', epoch_loss / len(dataloader))

# try to reconstruct the first sample based on the model
sample_input = data[0].unsqueeze(0).to(device)
reconstructed = model(sample_input)
print("original:")
print(sample_input)
print("reconstructed:")
print(reconstructed)

cuda
Epoch: 10 / 100  Loss: 0.19335379699865976
Epoch: 20 / 100  Loss: 0.19242097878897632
Epoch: 30 / 100  Loss: 0.19241185320748222
Epoch: 40 / 100  Loss: 0.19242376972127845
Epoch: 50 / 100  Loss: 0.19242376309854012
Epoch: 60 / 100  Loss: 0.19243269717251812
Epoch: 70 / 100  Loss: 0.19242406167365886
Epoch: 80 / 100  Loss: 0.19243369444652839
Epoch: 90 / 100  Loss: 0.1924355979318972
Epoch: 100 / 100  Loss: 0.19243286108529126
original:
tensor([[0.5553, 0.4815, 0.3130,  ..., 0.7352, 0.1472, 0.3634]],
       device='cuda:0')
reconstructed:
tensor([[0.5007, 0.0000, 0.0000,  ..., 0.4920, 0.5116, 0.4900]],
       device='cuda:0', grad_fn=<ReluBackward0>)
